In [2]:
# Yolo imports
from __future__ import division
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import cv2 
import time
import argparse
import os 
import os.path as osp
import pickle as pkl
import pandas as pd
import random
from skimage import io
from matplotlib import pyplot as plt
from ConfidenceThresholdingAndNonMaximumSuppression import *
from NeuralNetwork import *
from Utilities import *

#########################################################################################################################

# Lane detection and distance estimation imports
import glob
from skimage import io, data
import matplotlib.image as mpimg
import camera_calibration
import lane_detection
import vanishing_point
#import utilities
import perspective_transform

# YOLO code to be run only once ########################################################################################

#Driver values

images = 'imgs/'
batch_size = 1
confidence = 0.5
nms_thesh = 0.4
start = 0
weights_file = 'yolov3.weights'
cfg_file = 'cfg/yolov3.cfg'

# weights_file = 'yolov3-tiny.weights'
# cfg_file = 'cfg/yolov3-tiny.cfg'

#Change cfg file and save
def SetYoloReso(reso):
    with open(cfg_file, "r+") as f:
        lines = f.readlines()
        del lines[3]
        del lines[3]
        lines.insert(3, 'height = ' + str(reso) + '\n') 
        lines.insert(3, 'width = ' + str(reso) + '\n') 
        f.seek(0)
        f.truncate()
        f.writelines(lines)
        
reso = 32*7
SetYoloReso(reso)
det = 'det/'
CUDA = torch.cuda.is_available()

num_classes = 80    #For COCO
classes = load_classes("data/coco.names")

######

#Set up the neural network
print("Loading network.....")
model = Darknet(cfg_file)
model.load_weights(weights_file)
print("Network successfully loaded")

model.net_info["height"] = reso
inp_dim = int(model.net_info["height"])
assert inp_dim % 32 == 0 
assert inp_dim > 32

#If there's a GPU availible, put the model on GPU
if CUDA:
    model.cuda()

#Set the model in evaluation mode
model.eval()

#########################################################################################################################

# Lane detection and distance estimation code to be run only once

ret, mtx, dist, rvecs, tvecs = camera_calibration.calibrate(False)

#########################################################################################################################

# YOLO prediction function
# Takes BGR image and returns: 1) image annotated with bounding boxes, 2) list of top left and bottom right
# coordinates of bounding boxes, 3) list of class numbers which correspond to class labels

def get_pred(img):
    read_dir = time.time()
    #Detection phase
    imlist = ['img']

    if not os.path.exists(det):
        os.makedirs(det)

    load_batch = time.time()
    loaded_ims = [np.array(img)]
#     loaded_ims = [cv2.undistort(np.array(img), mtx, dist, None, mtx)]
    
    #PyTorch Variables for images
    im_batches = list(map(prep_image, loaded_ims, [inp_dim for x in range(len(imlist))]))

    #List containing dimensions of original images
    im_dim_list = [(x.shape[1], x.shape[0]) for x in loaded_ims]
    im_dim_list = torch.FloatTensor(im_dim_list).repeat(1,2)

    if CUDA:
        im_dim_list = im_dim_list.cuda()

    leftover = 0
    if (len(im_dim_list) % batch_size):
        leftover = 1

    if batch_size != 1:
        num_batches = len(imlist) // batch_size + leftover            
        im_batches = [torch.cat((im_batches[i*batch_size : min((i +  1)*batch_size, len(im_batches))]))  for i in range(num_batches)]  

    write = 0
    start_det_loop = time.time()
    
    for i, batch in enumerate(im_batches):
        #load the image 
        start = time.time()
        t = time.time()  
        if CUDA:
            batch = batch.cuda()
        with torch.no_grad():
            prediction = model(Variable(batch), CUDA)
#         print("prediction: " , time.time()-t)    
        t = time.time()    
        prediction = write_results(prediction, confidence, num_classes, nms_conf = nms_thesh)
        end = time.time()

        if type(prediction) == int:

            for im_num, image in enumerate(imlist[i*batch_size: min((i +  1)*batch_size, len(imlist))]):
                im_id = i*batch_size + im_num
#                 print("{0:20s} predicted in {1:6.3f} seconds".format(image.split("/")[-1], (end - start)/batch_size))
#                 print("{0:20s} {1:s}".format("Objects Detected:", ""))
#                 print("----------------------------------------------------------")
            continue

        prediction[:,0] += i*batch_size    #transform the atribute from index in batch to index in imlist 
    
    try:
        if not write:                      #If we have't initialised output
            output = prediction  
            write = 1
        else:
            output = torch.cat((output,prediction))

        for im_num, image in enumerate(imlist[i*batch_size: min((i +  1)*batch_size, len(imlist))]):
            im_id = i*batch_size + im_num
            objs = [classes[int(x[-1])] for x in output if int(x[0]) == im_id]
#             print("{0:20s} predicted in {1:6.3f} seconds".format(image.split("/")[-1], (end - start)/batch_size))
#             print("{0:20s} {1:s}".format("Objects Detected:", " ".join(objs)))
#             print("----------------------------------------------------------")

        if CUDA:
            torch.cuda.synchronize()
#         output
    except:
        #print("exception")
        return loaded_ims[0]

    im_dim_list = torch.index_select(im_dim_list, 0, output[:,0].long())

    scaling_factor = torch.min(inp_dim/im_dim_list,1)[0].view(-1,1)

    output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim_list[:,0].view(-1,1))/2
    output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim_list[:,1].view(-1,1))/2

    output[:,1:5] /= scaling_factor

    for i in range(output.shape[0]):
        output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim_list[i,0])
        output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim_list[i,1])

    class_load = time.time()
    colors = pkl.load(open("pallete", "rb"))

    draw = time.time()

    list(map(lambda x: write_img(x, loaded_ims, color = random.choice(colors), classes = classes), output))

    t = time.time()
    det_names = pd.Series(imlist).apply(lambda x: "{}/det_{}".format(det,x.split("/")[-1]))
    
    end = time.time()
    
#     print("SUMMARY")
###     print("----------------------------------------------------------")
###     print("{:25s}: {}".format("Task", "Time Taken (in seconds)"))
###     print()
###     print("{:25s}: {:2.3f}".format("Reading addresses", load_batch - read_dir))
###     print("{:25s}: {:2.3f}".format("Loading batch", start_det_loop - load_batch))
###     print("{:25s}: {:2.3f}".format("Drawing Boxes", end - draw))
#     print("{:25s}: {:2.3f}".format("Average time_per_img", (end - load_batch)/len(imlist)))
#     print("----------------------------------------------------------")

#     t = time.time()
#     torch.cuda.empty_cache()
#     print('= Time to torch.cuda.empty_cache : ', time.time() - t)
   
    return loaded_ims[0], output[:, 1:3], output[:, 3:5], output[:, -1]

#########################################################################################################################

# Perspective transform function.
# Takes as input: 1) BGR image, 2) debug mode, 3) YOLO annotated image
# Returns: 1) Homography matrix, 2) x pixels per meter, 3) y pixels per meter
# To be called only one time one time on a base image, its outputs are then used for any image

def driver_perspective_transform(img_BGR, debug=False):
#     ud_img_BGR = cv2.undistort(img_BGR, mtx, dist, None, mtx)
    ud_img_BGR = img_BGR
    ud_img_RGB = cv2.cvtColor(ud_img_BGR, cv2.COLOR_BGR2RGB)
    if(debug):
        show_images([ud_img_RGB])
    detector = lane_detection.LaneDetector()
    lines = detector.process(ud_img_RGB, True, 0.5, 0.16, debug)
    
    vp = vanishing_point.calculate_vanishing_point(lines, ud_img_BGR, debug)
    
    H, H_inv, warped = perspective_transform.perspective_transform(vp, ud_img_BGR, debug)
    
    x_pixels_per_meter , y_pixels_per_meter, left_low, left_high, right_low, right_high = \
                    perspective_transform.get_ratio(H, H_inv, warped, mtx, debug)

    return H, x_pixels_per_meter, y_pixels_per_meter

#########################################################################################################################

# Distance estimation function
# Takes as input: 1) Query pixel, 2) Bottom center coordinates of screen, 3) homography matrix,
# 4) x pixels per meter, 5) y pixels per meter
# Returns distance to the query pixel
def get_distance(query_pnt, center, H, x_pixels_per_meter, y_pixels_per_meter):
    return perspective_transform.get_distance(query_pnt, center, H, x_pixels_per_meter, y_pixels_per_meter)

#######################################################################################################################

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading network.....
Network successfully loaded


In [3]:
# Annotates image with lanes or prints on image that lane departure is detected
# Inputs: 1) image, 2) resolution of image / 1200x500 resolution
# Outputs: 2) Annotated image

def get_lane_image(img, ratio, debug=False):
    base_left_m, base_left_c, base_right_m, base_right_c = -0.32785089597153977, 446.0614062879984*ratio,\
                                                            0.2911316010810115, 79.62376483613377*ratio
    max_m_diff, max_c_diff = 0.13, 50

    test_img = np.array(img)
    width, height = test_img.shape[1], test_img.shape[0]

    detector = lane_detection.LaneDetector()
    lines = detector.process(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB), True, 0.4, 0.5, debug)

    left_lane_lines, right_lane_lines = lane_detection.get_lane_lines(lines, base_left_m, base_left_c, base_right_m,
                                                                      base_right_c, max_m_diff, max_c_diff)
    if(len(left_lane_lines)==0 or len(right_lane_lines)==0):
        inlane = True
        cv2.putText(test_img,
            "Not inside lane",
            (int(width/2-150), 50), 
            font, 
            fontScale,
            fontColor,
            lineType)
    else:
        inlane = False
        left_lane_line, right_lane_line = get_lines_mean(left_lane_lines), get_lines_mean(right_lane_lines)
        low_y, high_y = height-1, int(height * 0.6)
        left_low, left_high = (int((low_y-left_lane_line[1])/left_lane_line[0]), low_y),\
                              (int((high_y-left_lane_line[1])/left_lane_line[0]), high_y)
        right_low, right_high = (int((low_y-right_lane_line[1])/right_lane_line[0]), low_y),\
                              (int((high_y-right_lane_line[1])/right_lane_line[0]), high_y)
        cv2.line(test_img, tuple(left_low), tuple(left_high),(0,0,255),5)
        cv2.line(test_img, tuple(right_low), tuple(right_high),(0,0,255),5)
    
    return test_img, inlane

In [4]:
##Emergency braking fns
import math
import threading
import time    
# To play sounds
import winsound

last_t = time.time()
last_dist = 0
last_RV = 0
last_VDi = 0

Braking = False
BrakeStartTime = time.time()

def scale(val, src, dst):
    return ((val - src[0]) / (src[1]-src[0])) * (dst[1]-dst[0]) + dst[0]

def Brake3Secs():
    global Braking
    if(Braking):
        return
    Braking = True
    BADAS_fns.StartFullBrake()
    time.sleep(3)
    Braking = False
    BADAS_fns.StopBrake()

def BrakeSystemUpdate(new_dist,CurTime,BrakeStartTime):
    global Braking
    if(Braking and (new_dist > 15 or CurTime - BrakeStartTime > 0.5) ):
        print('StopBrake')
        BADAS_fns.StopBrake()
        Braking = False

inlane = False
WarningSnRunning = False
def WarningSn():
    global Braking
    global inlane
    global WarningSnRunning
    global veh_speed
    while(inlane):
        print('veh_speed',veh_speed)
        if(veh_speed > 5 and Braking == False):
            winsound.Beep(300, 800)
        time.sleep(0.5)
    WarningSnRunning = False

def LaneWarningControllerUpdate():
    global inlane
    global WarningSnRunning
    if(inlane and WarningSnRunning == False):
        WarningSnRunning = True
        threading.Thread(target=WarningSn).start()

def BrakeDecide(new_dist):
    global BrakeStartTime
    global Braking
    global last_RV
    global last_dist
    global last_VDi
    global last_t
    
    time_from_last_frame = time.time() - last_t
    RV =  ( last_dist - new_dist ) / time_from_last_frame
    RA = -( last_RV - RV) / time_from_last_frame
    
    VDi = veh_speed #BADAS_fns.client.getCarState().speed
    VOi = VDi - RV
    AD = (VDi - last_VDi) / time_from_last_frame
    AO = AD - RA
    
    ###Set last frame vals
    last_t = time.time()
    last_dist = new_dist
    last_RV = RV
    last_VDi = VDi
    
    max_decel = 20
    ds = ( VDi*VDi ) / (2*max_decel)
    ts = VDi / max_decel
    VOf = VOi + AO*ts
    if(VOf >= 0):
        do = ( VOi*ts ) + (0.5 * AO * (ts*ts))
    else:
        if(AO == 0):
            do = 1000
        else:
            do = -(VOi * VOi) / (2 * AO)

#     print('\ndistance = ' , new_dist )
#     print('time_from_last_frame = ' , time_from_last_frame )
#     print('RV = ' , RV )
#     print('RA = ' , RA )
#     print('VDi = ' , VDi )
#     print('VOi = ' , VOi )
#     print('AD = ' , AD )
#     print('AO = ' , AO )
#     print('ds = ' , ds )
#     print('ts = ' , ts )
#     print('do = ' , do )
#     print('(new_dist + do) - ds = ' , (new_dist + do) - ds )
    
    min_safe_dist = 8
    X = (new_dist + do) - ds
    
    if( X <= 0 ):
        prob = 1
    elif( X >= min_safe_dist ):
        prob = 0
    else:
        prob = scale(X , [0,min_safe_dist] , [1,0])
    
    print( 'prob: ' , prob )
    
    if( prob > 0.5 ):
#         Brake3Secs()
        if(Braking != True):
            Braking = True
            print('StartBrake')
            BADAS_fns.StartFullBrake()
            BrakeStartTime = time.time()
    else:
        BrakeSystemUpdate(new_dist, time.time(), BrakeStartTime)
        
#     print('rel time = ' , time.time() - t )
#     if VisionEmergencyCheck(RA,RV,new_dist):
#         threading.Thread(target=Brake2Secs).start()

def ThresholdCheck(T):
    print('T for collision: ', T)
    if(T <= 0.65 and T > 0):
#         if( T < 1 and T > 0.2 ):
#             ManueverRight()
        return True
    return False

# a: 0.5 * acc
# b: vel
# c: -distance
def VisionEmergencyCheck(acc, vel, dist):
    if( vel < 0.001):
        return False
    
    if( acc == 0 ):
        return ThresholdCheck(dist/vel)
    else:
        a = 0.5 * acc
        b = vel
        c = -dist
        
        d = (b**2) - (4*a*c)
        if(d<0):
#             print('No collision (d<0)')
            return False
        
        T = (-b+math.sqrt(d))/(2*a)
        return ThresholdCheck(T)

###################
# SimConnectAndCheck()
# while True:
#     time.sleep(0.2)
#     prob = BrakeDecide((BADAS_fns.client.getAdasPacket())[1])

In [7]:
##################################################################################################################################################################
# Importing Airsim
# Uses local airsim package (DO NOT pip install, UNINSTALL IF INSTALLED ALREADY: "pip uninstall airsim")
from pathlib import Path
import sys
import os
import time
from ctypes import *
sys.path.append(str(Path().resolve().parent.parent).replace('\\','/') + '/AirSimClient')
os.environ['PATH'] = str(Path().resolve().parent.parent).replace('\\','/') + '/AirSimClient' + os.pathsep + os.environ['PATH']
d = CDLL('SDL2.dll') #Steering kit haptic feedback dll
import BADAS_fns

##Change Airsim Img Res
from shutil import copy2
AirSimDocPath = os.path.expanduser('~/Documents') + '/AirSim'

def SetSimImgRes(resoW,resoH):
    AirSimDocPath = os.path.expanduser('~/Documents') + '/AirSim/settings.json'
    with open(AirSimDocPath , "r+") as f:
        lines = f.readlines()
        del lines[10]
        del lines[10]
        lines.insert(10, '"Height": ' + str(resoH) + '\n') 
        lines.insert(10, '"Width": ' + str(resoW) + ',\n') 
        f.seek(0)
        f.truncate()
        f.writelines(lines)
        
def GetSimImg():
    responses = BADAS_fns.client.simGetImages([ BADAS_fns.airsim.ImageRequest("0", BADAS_fns.airsim.ImageType.Scene, False, False) ])
    response = responses[0]
    img1d = np.fromstring(response.image_data_uint8, dtype=np.uint8) 
    img = img1d.reshape(response.height, response.width, 4)  
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

def SimConnectAndCheck():
    BADAS_fns.Find_BADAS_Client()
    time.sleep(0.5)
    BADAS_fns.client.confirmConnection()

#If changed, Sim must be restarted to take effect
SetSimImgRes(500,208)
# SetSimImgRes(1200,500)
# SetSimImgRes(1440,600)

No haptic devices found, Driving kit not connected.


In [ ]:
# Sim Real-time distance detection and showing output using CV2

#Connect/Reconnect
SimConnectAndCheck()

# Braking = False
##################################################################################################################################################################

def crop_center(img,cropx,cropy):
    y,x,z = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

# Preparing text style that will be used in writing distances
font                   = cv2.FONT_HERSHEY_SIMPLEX
pos = (0,0)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2

img = cv2.imread('imgs/noshade1200x500.JPG')
H, x_pixels_per_meter, y_pixels_per_meter = driver_perspective_transform(img, False)

#Calculate distance accuracy 
# dist_lst1 = []
# tott_lst = []

drawDebug = True
printDebug = True

while True:
    # Press "q" to quit
    if drawDebug and cv2.waitKey(25) & 0xFF == ord("q"):
        cv2.destroyAllWindows()
        break
    
    t3 = time.time()
    t1 = t3
    
    veh_speed = BADAS_fns.client.getCarState().speed
    
    try:
        img = GetSimImg()
        if(drawDebug):
            cv2.imshow("", img)
        if(printDebug):
            print('T Sim image extract: ' , time.time() - t1)
    except:
        print('GetSimImg Exception ')
        continue
    
    # Image bottom center coordinates
    center = [img.shape[1]//2, img.shape[0]-1]
    
    try:
        t1 = time.time()
        img, inlane = get_lane_image(img, float(img.shape[0])/500)
        LaneWarningControllerUpdate()
            
        if(printDebug):
            print('T get_lane_image: ' , time.time() - t1)
        
        imgYOLO = crop_center(img,208,208)
        
        if(drawDebug):
            cv2.imshow("", img)
        # Getting predictions from yolo
        t1 = time.time()
        pred, top_left, bottom_right, labels = get_pred(imgYOLO)
        if(printDebug):
            print('T Yolo: ' , time.time() - t1)
    except:
        print('YException ')
        continue
    
#     failed getting distance from yolo box size
#     BoxXdist = math.sqrt( (top_left[0][1] - top_left[0][0])**2 + (bottom_right[0][0] - bottom_right[0][1])**2 )
#     print('BoxXdist',BoxXdist)
    
    t1 = time.time()
    # Looping on every detected object
    for i in range(len(labels)):
        # Object label
        label = classes[int(labels[i])]

        if(label != "car"):
            continue

        # Top left x,y
        tlx, tly = int(top_left[i,0]), int(top_left[i,1])

        # Bottom right x,y
        brx, bry = int(bottom_right[i,0]), int(bottom_right[i,1])

        # Bottom center coordinates of bounding box
        cx, cy = (tlx + brx)//2, bry

        #print(str(cx) + "," + str(cy) + " | " + str(center[0]) + "," + str(center[1]))

        # Distance to car
        float_dist = get_distance(scale_pixel([cx, cy], img.shape), scale_pixel(center, img.shape), H, x_pixels_per_meter, y_pixels_per_meter)

        #Mapping distance
#         float_dist = scale(float_dist, (3.1587, 31) , (3.7459, 27.7)) #From height500 imgs

        EmergencyBool = BrakeDecide(float_dist)
        
#         try:
#             # Extract distance from sim #########################
#             actual_dist = BADAS_fns.client.getAdasPacket()[1]
#             dist_lst1.append( abs(actual_dist - float_dist) )
#             print('actual dist: ' , actual_dist)
#             print('estimated dist: ' , float_dist)
#         except:
#             print('Extract distance from sim except')
#             pass
        ############################################################
        if(drawDebug):
            dtime = time.time()
            #paste yolo img on lane img
            img[0:pred.shape[0] , 146:146 + pred.shape[1]] = pred
#             img[0:pred.shape[0] , 350:350 + pred.shape[1]] = pred
            distance = str(round_float(float_dist))+"m"
            if(EmergencyBool):
                cv2.putText(img, "EMERGENCY",(cx-60+146, cy-60), font, fontScale,fontColor, lineType)
           # Annotate yolo image with distance to car
            cv2.putText(img, distance, (cx-60+146, cy+30), font, fontScale, fontColor, lineType)
            if(printDebug):
                print('T Get distance: ' , time.time() - t1)
            #print(x_pixels_per_meter)
            #print(y_pixels_per_meter)
            cv2.imshow("", img)
            if(printDebug):
                print('T Debug Draw: ' , time.time() - dtime)
#     tott_lst.append(time.time() - t3)
    if(printDebug):
        print('T Total: ' , time.time() - t3 , '\n')
        print('FrameEnd\n')

# print('num: ' , len(dist_lst1))
# print("dist mean absolute error: ", sum(dist_lst1) / len(dist_lst1) )
# print("Tott avg: ", sum(tott_lst) / len(tott_lst) )

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

pxls/mtr: 33.35520666666667,45.70352015283409
T Sim image extract:  0.21086931228637695
T get_lane_image:  0.003998279571533203
T Yolo:  0.09148979187011719
prob:  0
actual dist:  20.060514450073242
estimated dist:  29.634014615298774
T Get distance:  0.011993169784545898
T Debug Draw:  0.0029981136322021484
T Total:  0.3353402614593506 

FrameEnd

T Sim image extract:  0.009993553161621094
T get_lane_image:  0.005995750427246094
T Yolo:  0.08994531631469727
prob:  0
actual dist:  19.709291458129883
estimated dist:  29.634014615298774
T Get distance:  0.01099252700805664
T Debug Draw:  0.002997875213623047
T Total:  0.14091277122497559 

FrameEnd

T Sim image extract:  0.00899505615234375
T get_lane_image:  0.00499725341796875
T Yolo:  0.08994483947753906
prob:  0
actual dist:  19.709291458129883
estimated dist:  25.42664447545981
T Get distance:  0.010993480682373047
T Debug Draw:  0.004996776580810547
T Total:  0.135916

T Debug Draw:  0.00099945068359375
T Total:  0.14527392387390137 

FrameEnd

T Sim image extract:  0.008995771408081055
T get_lane_image:  0.004996061325073242
T Yolo:  0.1169288158416748
prob:  0
actual dist:  19.995502471923828
estimated dist:  23.1827162270683
T Get distance:  0.010993480682373047
T Debug Draw:  0.003996610641479492
T Total:  0.15590429306030273 

FrameEnd

T Sim image extract:  0.00799417495727539
T get_lane_image:  0.0049970149993896484
YException 
T Sim image extract:  0.00899505615234375
T get_lane_image:  0.004996776580810547
T Yolo:  0.102935791015625
prob:  0
actual dist:  20.040687561035156
estimated dist:  21.271219666411938
T Get distance:  0.012992382049560547
T Debug Draw:  0.003997325897216797
T Total:  0.14591026306152344 

FrameEnd

T Sim image extract:  0.007996797561645508
T get_lane_image:  0.00499725341796875
T Yolo:  0.10393619537353516
prob:  0
actual dist:  20.026687622070312
estimated dist:  23.1827162270683
T Get distance:  0.0149908065795898

T Sim image extract:  0.008995532989501953
T get_lane_image:  0.0059967041015625
T Yolo:  0.1209251880645752
prob:  0.3463804058617066
actual dist:  3.824815034866333
estimated dist:  4.16836139591507
T Get distance:  0.011992931365966797
T Debug Draw:  0.005995988845825195
T Total:  0.16489863395690918 

FrameEnd

T Sim image extract:  0.007995367050170898
T get_lane_image:  0.004997730255126953
T Yolo:  0.11292910575866699
T Total:  0.1369161605834961 

FrameEnd

T Sim image extract:  0.012989044189453125
T get_lane_image:  0.0069959163665771484
T Yolo:  0.1189260482788086
T Total:  0.15190505981445312 

FrameEnd

T Sim image extract:  0.007995367050170898
T get_lane_image:  0.004996538162231445
T Yolo:  0.10793375968933105
T Total:  0.13191819190979004 

FrameEnd

T Sim image extract:  0.01399087905883789
veh_speedT get_lane_image:  10.5595703125
 0.005997896194458008
YException 
T Sim image extract:  0.017989635467529297
T get_lane_image:  0.003998279571533203
T Yolo:  0.1109313964

T Sim image extract:  0.008996248245239258
T get_lane_image:  0.004996538162231445
T Yolo:  0.11592864990234375
prob:  0
actual dist:  20.473922729492188
estimated dist:  25.42664447545981
T Get distance:  0.007994890213012695
T Debug Draw:  0.003997802734375
T Total:  0.15490484237670898 

FrameEnd

T Sim image extract:  0.017988920211791992
T get_lane_image:  0.00499725341796875
T Yolo:  0.11358809471130371
prob:  0
actual dist:  20.68036651611328
estimated dist:  33.21806749481543
T Get distance:  0.015990734100341797
T Debug Draw:  0.0049970149993896484
T Total:  0.1695549488067627 

FrameEnd

T Sim image extract:  0.007995367050170898
T get_lane_image:  0.005995512008666992
T Yolo:  0.11193156242370605
prob:  0
actual dist:  20.5689697265625
estimated dist:  33.21806749481543
T Get distance:  0.00999307632446289
T Debug Draw:  0.003998517990112305
T Total:  0.15190577507019043 

FrameEnd

T Sim image extract:  0.01099395751953125
T get_lane_image:  0.003998517990112305
T Yolo:  0.

YException 
T Sim image extract:  0.013992547988891602
T get_lane_image:  0.004997730255126953
YException 
T Sim image extract:  0.007995367050170898
T get_lane_image:  0.005997657775878906
YException 
T Sim image extract:  0.008996248245239258
T get_lane_image:  0.0050013065338134766
YException 
T Sim image extract:  0.011993169784545898
T get_lane_image:  0.00499725341796875
T Yolo:  0.10793209075927734
prob:  0
actual dist:  23.75655746459961
estimated dist:  25.42664447545981
T Get distance:  0.009994029998779297
T Debug Draw:  0.003997325897216797
T Total:  0.15390539169311523 

FrameEnd

T Sim image extract:  0.00899505615234375
T get_lane_image:  0.004995584487915039
T Yolo:  0.1089327335357666
prob:  0
actual dist:  21.084665298461914
estimated dist:  25.42664447545981
T Get distance:  0.013991594314575195
T Debug Draw:  0.0029985904693603516
T Total:  0.15090727806091309 

FrameEnd

T Sim image extract:  0.008995532989501953
T get_lane_image:  0.0049970149993896484
T Yolo:  0.

T Get distance:  0.012991666793823242
T Debug Draw:  0.002997875213623047
T Total:  0.1229240894317627 

FrameEnd

T Sim image extract:  0.007995128631591797
T get_lane_image:  0.004996776580810547
T Yolo:  0.08394885063171387
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.008994579315185547
T Debug Draw:  0.001998424530029297
T Total:  0.1169271469116211 

FrameEnd

T Sim image extract:  0.008994817733764648
T get_lane_image:  0.0039975643157958984
T Yolo:  0.08351254463195801
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.01099252700805664
T Debug Draw:  0.0029976367950439453
T Total:  0.11949014663696289 

FrameEnd

T Sim image extract:  0.00799560546875
T get_lane_image:  0.0039975643157958984
T Yolo:  0.08394813537597656
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.011992692947387695
T Debug Draw:  0.002997875213623047
T Total:  0.121925

T Get distance:  0.008937835693359375
T Debug Draw:  0.00299835205078125
T Total:  0.1241767406463623 

FrameEnd

T Sim image extract:  0.008994579315185547
T get_lane_image:  0.004997730255126953
T Yolo:  0.08894467353820801
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.01099395751953125
T Debug Draw:  0.0029973983764648438
T Total:  0.12792110443115234 

FrameEnd

T Sim image extract:  0.008994817733764648
T get_lane_image:  0.0039975643157958984
T Yolo:  0.08294892311096191
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.011993646621704102
T Debug Draw:  0.002997875213623047
T Total:  0.1229248046875 

FrameEnd

T Sim image extract:  0.007995128631591797
T get_lane_image:  0.004997730255126953
T Yolo:  0.08594727516174316
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.008995532989501953
T Debug Draw:  0.0019979476928710938
T Total:  0.120925

estimated dist:  56.025690811120555
T Get distance:  0.009993553161621094
T Debug Draw:  0.0029981136322021484
T Total:  0.12492823600769043 

FrameEnd

T Sim image extract:  0.007995367050170898
T get_lane_image:  0.0039980411529541016
T Yolo:  0.08594655990600586
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.009994029998779297
T Debug Draw:  0.0019991397857666016
T Total:  0.1209259033203125 

FrameEnd

T Sim image extract:  0.00799560546875
T get_lane_image:  0.0069959163665771484
T Yolo:  0.08394765853881836
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.008994340896606445
T Debug Draw:  0.00299835205078125
T Total:  0.11992669105529785 

FrameEnd

T Sim image extract:  0.007994890213012695
T get_lane_image:  0.0059964656829833984
T Yolo:  0.08294820785522461
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.011993169784545898
T Debug Draw:  

estimated dist:  56.025690811120555
T Get distance:  0.0099945068359375
T Debug Draw:  0.0019989013671875
T Total:  0.12092471122741699 

FrameEnd

T Sim image extract:  0.007995128631591797
T get_lane_image:  0.004997730255126953
T Yolo:  0.08394813537597656
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.011993646621704102
T Debug Draw:  0.0029976367950439453
T Total:  0.12192535400390625 

FrameEnd

T Sim image extract:  0.008995532989501953
T get_lane_image:  0.004996776580810547
T Yolo:  0.08494734764099121
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.012991666793823242
T Debug Draw:  0.003998279571533203
T Total:  0.12592267990112305 

FrameEnd

T Sim image extract:  0.007996559143066406
T get_lane_image:  0.004996776580810547
T Yolo:  0.08694577217102051
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.010994195938110352
T Debug Draw:  0.

estimated dist:  56.025690811120555
T Get distance:  0.009993791580200195
T Debug Draw:  0.0019986629486083984
T Total:  0.12183523178100586 

FrameEnd

T Sim image extract:  0.008994102478027344
T get_lane_image:  0.005995750427246094
T Yolo:  0.08518338203430176
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.00999307632446289
T Debug Draw:  0.0029993057250976562
T Total:  0.12215924263000488 

FrameEnd

T Sim image extract:  0.008995532989501953
T get_lane_image:  0.005996227264404297
T Yolo:  0.08394789695739746
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.0099945068359375
T Debug Draw:  0.004996538162231445
T Total:  0.1229245662689209 

FrameEnd

T Sim image extract:  0.008994340896606445
T get_lane_image:  0.003997802734375
T Yolo:  0.08394742012023926
prob:  0
actual dist:  20.3071231842041
estimated dist:  56.025690811120555
T Get distance:  0.009994268417358398
T Debug Draw:  0.00

In [ ]:
# Reading image
img = cv2.imread('imgs/noshade1200x500.JPG')

print(img.shape)

# Image bottom center coordinates
center = [img.shape[1]//2, img.shape[0]-1]

# Preparing text style that will be used in writing distances
font                   = cv2.FONT_HERSHEY_SIMPLEX
pos = (0,0)
fontScale              = 1
fontColor              = (0,0,0)
lineType               = 2

# Getting predictions from yolo
pred, top_left, bottom_right, labels = get_pred(img)
    
# Perform perspective transform and annotate yolo image
H, x_pixels_per_meter, y_pixels_per_meter = driver_perspective_transform(img, True)

# Looping on every detected object
for i in range(len(labels)):
    
    # Object label
    label = classes[int(labels[i])]
    
    if(label != "car"):
        continue
        
    # Top left x,y
    tlx, tly = int(top_left[i,0]), int(top_left[i,1])
    
    # Bottom right x,y
    brx, bry = int(bottom_right[i,0]), int(bottom_right[i,1])
        
    # Bottom center coordinates of bounding box
    cx, cy = (tlx + brx)//2, bry
    
    print(str(cx) + "," + str(cy) + " | " + str(center[0]) + "," + str(center[1]))
    
    # Distance to car
    float_dist = get_distance([cx, cy], center, H, x_pixels_per_meter, y_pixels_per_meter)

    distance = str(round_float(float_dist))+"m"
    
    # Annotate yolo image with distance to car
    cv2.putText(pred,
        distance,
        (cx-60, cy+30), 
        font, 
        fontScale,
        fontColor,
        lineType)

print(x_pixels_per_meter)
print(y_pixels_per_meter)
    
show_images([cv2.cvtColor(pred, cv2.COLOR_BGR2RGB)])

In [ ]:
test_img1 = cv2.imread('imgs/1202x498.jpg')
out_img1 = get_lane_image(test_img1, 1)
show_images([cv2.cvtColor(out_img1, cv2.COLOR_BGR2RGB)])

test_img2 = cv2.imread('imgs/noshade500x208.jpg')
out_img2 = get_lane_image(test_img2, 500/1200.0)
show_images([cv2.cvtColor(out_img2, cv2.COLOR_BGR2RGB)])


In [ ]:
cnt = 0
directory = os.fsencode("images1")
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".jpg"):
        test_img = cv2.imread("images1/"+filename)
        out_img = get_lane_image(test_img, float(test_img.shape[0])/500)
        cv2.imwrite("out_images1/out"+str(cnt)+".jpg", out_img)
        cnt += 1

In [ ]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

#BADAS v2.6+
def SteerByAngle(angle, direction):
    
    FirstVec = BADAS_fns.client.getSuvFwdVec() #Get first forward vector
    
    car_controls = BADAS_fns.airsim.CarControls()
    
    car_controls.throttle = 0 # Change later, need to meet certain velocity depending on manuever equation
    
    if(direction == 'right'):
        car_controls.steering = 1
    elif(direction == 'left'):
        car_controls.steering = -1
        
    #Start steering    
    BADAS_fns.client.enableApiControl(True)  
    BADAS_fns.client.setCarControls(car_controls)
    
    #Keep steering until required angle is met
    while (True):
        SecVec = BADAS_fns.client.getSuvFwdVec()
        radians = angle_between( (FirstVec['X'], FirstVec['Y']), (SecVec['X'], SecVec['Y'])) #Get angle betwen first and second forward vectors
        
        if(math.degrees(radians) >= angle): #Stop steering if required angle is met
            break
        
        if(BADAS_fns.client.getCarState().speed < 1): #Stop steering in case of errors
            break
            
        time.sleep(0.01)
        
    BADAS_fns.client.enableApiControl(False) # Return driver full controls

def ManueverRight():
    SteerByAngle(20,'right')
    time.sleep(0.1)   
    SteerByAngle(20,'left')

In [ ]:
try:
    while True:
        #starttime = time.time()
        t = time.time()
        responses = client.simGetImages([ airsim.ImageRequest("0", airsim.ImageType.Scene, False, False) ])
        response = responses[0]
        img1d = np.frombuffer(response.image_data_uint8, dtype=np.uint8) 
        img_rgba = img1d.reshape(response.height, response.width, 4)  
        img_rgba = cv2.resize(img_rgba, dsize=(256, 128), interpolation=cv2.INTER_NEAREST) #####################RESIZING FOR FAST PREDICTION
        print('= T Extract sim image: ' , time.time()-t)

        t = time.time()
        detected = get_pred(img_rgba)
        print('= T Detecting: ' , time.time()-t)
        
        cv2.imshow("img", detected)
        #print('=== TOTAL TIME ' , time.time()-starttime , ' ===\n')
        
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            break
except:
    cv2.destroyAllWindows()

In [ ]:
import time
x = 100
while True:
    x = x -1
    if x < 0:
        break
    t = time.time()
    png_image = client.simGetImage("1", airsim.ImageType.Scene)
    print('time: ' , time.time()-t)

In [ ]:
responses = BADAS_fns.client.simGetImages([ BADAS_fns.airsim.ImageRequest("0", BADAS_fns.airsim.ImageType.Scene, False, False) ])
response = responses[0]
img1d = np.fromstring(response.image_data_uint8, dtype=np.uint8) 
img = img1d.reshape(response.height, response.width, 4)  
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

In [ ]:
#Print average of time to detect img and predict
timeout = time.time() + 3 #5 secs

lst = [] 
lst2 = [] 

while True:
    if time.time() > timeout:
        break
    start = time.time()
    responses = BADAS_fns.client.simGetImages([ BADAS_fns.airsim.ImageRequest("0", BADAS_fns.airsim.ImageType.Scene, False, False) ])
    response = responses[0]
    img1d = np.fromstring(response.image_data_uint8, dtype=np.uint8) 
    img = img1d.reshape(response.height, response.width, 4)  
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    print("T SimGetImg =", time.time() - start ) 
    lst2.append(time.time() - start)
    
    start = time.time()
    detected = get_pred(img)
    print("T Yolo =", time.time() - start ) 
    lst.append(time.time() - start)
        
print("AverageGetImage =", sum(lst2) / len(lst2) ) 
print("AverageYolo =", sum(lst) / len(lst) ) 

In [ ]:
#40fps fullscreen quality3 AverageAll = 0.105, AverageGetImage = 0.0021

#over lan 35fps fullscreen quality3
#AverageGetImage = 0.019696117627738725
#AverageAll = 0.09919774414289116

In [ ]:
import time
timeout = time.time() + 5 #5 secs

while True:
    if time.time() > timeout:
        break
    x = 0
    start = time.time()
    while True:
        responses = client.simGetImages([ airsim.ImageRequest("0", airsim.ImageType.Scene, False, False) ])
        response = responses[0]
        #detected = get_pred((np.frombuffer(response.image_data_uint8, dtype=np.uint8) ).reshape(response.height, response.width, 4))
        if(time.time() - start > 1):
            break
        x += 1

    print(x)

In [ ]:
#SHOW ONE IMAEG
png_image = BADAS_fns.client.simGetImage("0", BADAS_fns.airsim.ImageType.Scene)
Image.open(io.BytesIO(png_image)).show()

In [ ]:
import mss
import mss.tools
import time
from PIL import Image

while True:
    
    
    with mss.mss() as sct:
        for num, monitor in enumerate(sct.monitors[1:], 1):
            # Get raw pixels from the screen
            sct_img = sct.grab(monitor)
            # Create the Image
            img = Image.frombytes("RGB", sct_img.size, sct_img.bgra, "raw", "BGRX")

        # Grab the data
        t = time.time()
        sct_img = sct.grab(monitor)
        print(time.time() - t)
        #sct_img.
        #print(type(sct_img))
        # Save to the picture file
        #mss.tools.to_png(sct_img.rgb, sct_img.size, output=output)
        #print(output)

In [ ]:
responses = client.simGetImages([ airsim.ImageRequest("0", airsim.ImageType.Scene, False, False) ])
response = responses[0]
img1d = np.fromstring(response.image_data_uint8, dtype=np.uint8) 
img_rgba = img1d.reshape(response.height, response.width, 4)  
img_RGB = cv2.cvtColor(img_rgba, cv2.COLOR_RGB2BGR)
cv2.imwrite('3.jpg',img_RGB)

In [ ]:
def GetMonitorImg():
    with mss.mss() as sct:
        for num, monitor in enumerate(sct.monitors[1:], 1):
            # Get raw pixels from the screen
            sct_img = sct.grab(monitor)
            # Create the Image
            return Image.frombytes("RGB", sct_img.size, sct_img.bgra, "raw", "BGRX")

In [ ]:
def GetMonitorImg2():
    with mss.mss() as sct:
        monitor = {"top": 0, "left": 0, "width": 1920, "height": 1040}
        img = numpy.array(sct.grab(monitor))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        return img

In [ ]:
t = time.time()
get_pred(img)
print(time.time() - t)

In [ ]:
img = cv2.imread('imgs/noshade500x208.JPG')
# H, x_pixels_per_meter, y_pixels_per_meter = driver_perspective_transform(img, True)

In [ ]:
BADAS_fns.StopBrake()

In [ ]:
y = 50
h = 500

x= 100
w= 300

print(img.shape)
newim = img[y:y+h, x:x+w]
print(newim.shape)
cv2.imshow("cropped", newim)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img = cv2.imread('imgs/noshade1200x500.JPG')

y,x,z = img.shape
cropx = 320
startx = x//2-(cropx//2)
starty = y
img = img[y - 420: y,startx:startx+cropx]

cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img = cv2.imread('imgs/noshade1200x500.JPG')
y,x,z = img.shape
# img = img[y - 500: y, x - 700: x]

t =time.time() 
pred, top_left, bottom_right, labels = get_pred(img)
print(time.time() - t)

In [83]:
# imgYOLO = crop_center(img,600,600)
cv2.imshow("img", imgYOLO)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
pred, top_left, bottom_right, labels = get_pred(imgYOLO)